# Quality standards

## Imports

In [1]:
from weldx.config import Config, QualityStandard
import fs
import yaml
from weldx.measurement import  GenericEquipment

## Introduction

The main purposes of WelDX is to provide a file format for experimental data that is self explanatory and contains all
relevant information about the conducted experiment.
This allows fellow researchers to understand and analyze the data without taking part in the experiment or having access
to the persons who did.
We assure this by introducing quality standards for each type of experiment and every other information that
describes certain aspects of an experiment like coordinate systems, sensors or specimens.
Every time a WelDX file is written or read, it is validated against those standards.

## Schema files

Every standard used by WelDX is defined by schema files.
A schema file is a [YAML](https://yaml.org/)-file that defines the allowed properties of an serializable object.
How such a schema file is generated is not part of this tutorial, but you can find further information regarding this
topic in the official
[documentation of the ASDF standard](https://asdf-standard.readthedocs.io/en/latest/schemas.html) which is the
foundation of a WelDX file.
Furthermore, an overview of all pre-defined schema files can be found on the [Standards](../standard.rst) page of this
documentation.

> todo: fix link

## Installing and using quality standards

The WelDX API comes with a default schema for every object.
So if you save an ASDF file using the WelDX extension you already employ the default WelDX quality standard.
However, the standards we defined might not suit your needs.
Therefore, the WelDX API offers a mechanism to override an arbitrary number of default schemas.
Before we discuss, how a custom standard can be generated, let's assume that you have found and downloaded one.
Each quality standard that was created following the guidelines (Todo: Link) can be installed using pythons package
manager `pip`.
So the first step would be to install the standard as a python package:

~~~ shell
pip install NAME_OF_THE_STANDARD
~~~

Installing the quality standard registers it to the WelDX API so that it knows that the standard exists and where to
find the corresponding data.
If you want to use the standard, you have to activate it first:

~~~ python
from weldx.config import Config

Config.enable_quality_standard(NAME_OF_THE_STANDARD, VERSION_OF_THE_STANDARD)
~~~

The version number is optional.
If you don't provide one, the latest available version will be used.
And that's all there is to know about using standards.

## Creating custom standards

Creating custom standards is fairly easy if you already know how a schema file looks like.
To keep things simple, we won't discuss here how you can provide an installable standard because this might be a bit
confusing at the beginning.
Instead we will just focus on the files and their content.

Let's say we want to set a new quality standard for the `GenericEquipment`.
In our short example, we will use the following file structure:

In [2]:
mem_fs = fs.open_fs('mem://')
mem_fs.makedirs("resources/my_organization/manifests")
mem_fs.makedirs("resources/my_organization/schemas")
mem_fs.listdir("")

mem_fs.create("resources/my_organization/manifests/my_standard-1.0.0.yaml")
mem_fs.create("resources/my_organization/schemas/my_schema-1.0.0.yaml")
mem_fs.tree()

`-- resources
    `-- my_organization
        |-- manifests
        |   `-- my_standard-1.0.0.yaml
        `-- schemas
            `-- my_schema-1.0.0.yaml


The directory structure is based on how an installable standard has to be organized but the only thing that really
matters for us at the moment is the content of the `my_organization` directory.
It is subdivided into `manifests` and `schemas`.
All of your custom schemas go into the `schemas` directory.
The `manifests` directory contains so called manifest files.
Their purpose is to manage which schema belongs to which version of your standard.
All manifests are YAML files and must be named like your standard with the corresponding version number attached to the
end.
Here we will only use a single manifest file called `my_standard-1.0.0.yaml`.

Now before we start looking into the different files, let's create an instance of the `GenericEpuipment` class that we
want to serialize using a new quality standard:

In [3]:
my_equipment = GenericEquipment("my_equipment")

In [4]:
manifest = """
id: http://stsci.edu/asdf/extensions/transform/manifests/transform-1.0.0
extension_uri: http://stsci.edu/asdf/extensions/transform-1.0.0
title: Transform extension 1.0.0
description: |-
  A set of tags for serializing data transforms.
asdf_standard_requirement: 1.0.0
tags:
- uri: "http://weldx.bam.de/schemas/weldx/core/transformations/local_coordinate_system-1.0.0"
  file: "lcs_schema_fancy"
"""

In [5]:
with mem_fs.open("resources/my_organization/manifests/my_standard-1.0.0.yaml", "w") as file:
    file.write(manifest)

In [6]:
with mem_fs.open("resources/my_organization/manifests/my_standard-1.0.0.yaml", "r") as file:
    content = yaml.load(file, Loader=yaml.SafeLoader)
    print(content)

{'id': 'http://stsci.edu/asdf/extensions/transform/manifests/transform-1.0.0', 'extension_uri': 'http://stsci.edu/asdf/extensions/transform-1.0.0', 'title': 'Transform extension 1.0.0', 'description': 'A set of tags for serializing data transforms.', 'asdf_standard_requirement': '1.0.0', 'tags': [{'uri': 'http://weldx.bam.de/schemas/weldx/core/transformations/local_coordinate_system-1.0.0', 'file': 'lcs_schema_fancy'}]}


In [7]:
qs = QualityStandard(mem_fs.opendir("resources/my_organization"))